# Autocombat TOP+StrokeMRI, SABREand Insight46 harmonized datasets

This notebook creates datasets from the above hamornized as if each of 1. a blended TOP and StrokeMRI 2.SABRE and 3. Insight  is indpendant and from a different "batch" (TOP and StrokeMRI are same acquisition on same machine, so same batch)

Note this must be run in the `comscani` environment

## import libraries

In [1]:
import os

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 
filename_sabre = os.path.join(filepath,'SABRE_pvc2_cleaned.csv') 
filename_insight46 =  os.path.join(filepath,'Insight46_pvc2c.csv') 

In [ ]:
# Now we must clean SABRE down to be like the other datasets, including dropping NAns
MRI = pd.read_csv(filename_mri)
TOP = pd.read_csv(filename_top)
Insight46 = pd.read_csv(filename_insight46)

SABRE =  pd.read_csv(filename_sabre)
coly = TOP.columns
SABRE = SABRE[coly]
SABRE= SABRE.dropna()
SABRE.tail(3)